In [1]:
import numpy as np
import pandas as pd
import os
os.chdir("/content/drive/MyDrive/Domain Adaptation Project/Robust self-training/dataset")

In [17]:
labeled_df = pd.read_csv("./Labeled.csv")
unlabeled_df = pd.read_csv("./Unlabeled.csv")
val_df = pd.read_csv("./Val.csv")

In [24]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
labeled_df["num_tokens"] = labeled_df.apply(lambda row : len(word_tokenize(row["review/text"])), axis = 1)
print(len(labeled_df))
labeled_df_sub = labeled_df[(labeled_df['num_tokens'] > 10) & (labeled_df['num_tokens'] < 200)]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


6752
6124


In [26]:
labeled_df_sub_balanced = labeled_df_sub.groupby('is_helpful?', group_keys=False).apply(lambda x: x.sample(885))

In [27]:
labeled_df_sub_balanced['is_helpful?'].value_counts()

0    885
1    885
Name: is_helpful?, dtype: int64

In [28]:
labeled_df_sub_balanced = labeled_df_sub_balanced[['review/text', 'is_helpful?']]

In [31]:
labeled_df_sub_balanced.rename(columns={"review/text": "comments", "is_helpful?": "Helpful"},inplace = True)

In [35]:
import re
def clean_text(text):
  text = re.sub(r'\[[0-9]*\]',' ',text) # [0-9]* --> Matches zero or more repetitions of any digit from 0 to 9
  text = text.lower() #everything to lowercase
  text = re.sub(r'\W^.!?',' ',text) # \W --> Matches any character which is not a word character except (.?!)
  cleanr = re.compile('<.*?>')
  text = re.sub(cleanr, '', text)
  text = re.sub(r'\d',' ',text) # \d --> Matches any decimal digit
  text = re.sub(r'\s+',' ',text) # \s --> Matches any characters that are considered whitespace (Ex: [\t\n\r\f\v].)
  return text

In [34]:
labeled_df_sub_balanced.reset_index(drop=True, inplace=True)

In [36]:
labeled_df_sub_balanced["comments"] = labeled_df_sub_balanced["comments"].apply(clean_text)

In [38]:
labeled_df_sub_balanced = labeled_df_sub_balanced.sample(frac=1)

In [40]:
labeled_df_sub_balanced.reset_index(drop=True, inplace=True)

In [42]:
labeled_df_sub_balanced.to_csv("./Labeled_new.csv",index=False)